In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from sklearn.linear_model import LogisticRegression
from google.colab import files
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

train_data, train_labels, test_data, valid_data, valid_labels = [], [], [], [], []
with open ('/content/gdrive/My Drive/Машобуч ДЗ1/train_texts_2', 'r') as f1:
  for line in f1:
    train_data.append(line.strip())
with open('/content/gdrive/My Drive/Машобуч ДЗ1/test_texts_2', 'r') as f2:
  for line in f2:
    test_data.append(line.strip())
with open('/content/gdrive/My Drive/Машобуч ДЗ1/train_labels_2', 'r') as f3:
  for line in f3:
    train_labels.append(line.strip())
with open('/content/gdrive/My Drive/Машобуч ДЗ1/train_texts', 'r') as f4:
  for line in f4:
    valid_data.append(line.strip())
with open('/content/gdrive/My Drive/Машобуч ДЗ1/train_labels', 'r') as f4:
  for line in f4:
    valid_labels.append(line.strip())

In [ ]:
!pip install spacy[lookups]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.0 MB/s eta 0:00:00


In [ ]:
import spacy

model = spacy.blank("en")
model.add_pipe("lemmatizer", config={"mode": "lookup"})
model.initialize()

In [ ]:
from collections import Counter, defaultdict
from nltk.tokenize import wordpunct_tokenize
from string import punctuation as string_punctuation

def process_text(text, lowercase=False, remove_punctuation=False, normalize_digits=False):
    if isinstance(text, str):
        tokens = wordpunct_tokenize(text)
    else:
        tokens = text
    if lowercase:
        tokens = [token.lower() for token in tokens]
    if remove_punctuation:
        tokens = [token for token in tokens
                  if any(x not in string_punctuation for x in token)]
    if normalize_digits:
        tokens = ["<DIGIT>" if token.isdigit() else token for token in tokens]
    return Counter(tokens)

In [ ]:
from scipy import stats

class TextCounter:
    """
    Преобразует текст в словарь (индекс слова-счётчик), убирая редкие слова
    """

    def __init__(self, min_count=3, max_count=None, lemmatizer=None,
                 lowercase=False, remove_punctuation=False,
                 normalize_digits=False):
        self.min_count = min_count
        self.max_count = max_count
        self.lowercase = lowercase
        self.remove_punctuation = remove_punctuation
        self.normalize_digits = normalize_digits

    def __len__(self):
        return len(self.words_) if hasattr(self, "words_") else 0

    def fit(self, texts):
        if self.max_count is None:
            max_count = len(texts)
        elif self.max_count < 1:
            max_count = len(texts) * self.max_count # доля -> абсолютное значение
        else:
            max_count = self.max_count
        # обучаем словарь, собирая статистику по корпусу текстов
        word_counts = defaultdict(int)
        words_in_texts = dict()
        for i, text in enumerate(texts):
            text_word_counts = process_text(
                text, lowercase=self.lowercase,
                remove_punctuation=self.remove_punctuation,
                normalize_digits=self.normalize_digits)
            for word, count in text_word_counts.items():
                word_counts[word] += 1
        for word in word_counts:
          if word not in words_in_texts:
            words_in_texts = [word_counts[word]]
          else:
            words_in_texts.append(word_counts[word])
        words_uniform = dict()
        for word in words_in_texts:
          data = words_in_texts[word]
          stats, p_value = stats.kstest(data, 'uniform')
          if p_value < 0.05:
            words_uniform[word] = False
          else:
            words_uniform[word] = True
        self.words_ = [word for word, count in word_counts.items()
                       if count >= self.min_count and count <= max_count and not words_uniform[word]]
        self.word_indexes_ = {word: index for index, word in enumerate(self.words_)}
        self.word_counts_ = {
            index: word_counts[word] for index, word in enumerate(self.words_)}
        return self

    def transform(self, text, is_processed=False):
        word_counts = process_text(text, lowercase=self.lowercase,
                                   remove_punctuation=self.remove_punctuation,
                                   normalize_digits=self.normalize_digits)
        return {self.word_indexes_[word]:count for word, count in word_counts.items()
                if word in self.word_indexes_}

In [ ]:
from scipy import stats
data = [1, 2, 3, 4, 1, 2, 3, 5, 6, 3, 2, 3, 5, 6, 9, 10, 19, 13, 12]
#stat, p_value = stats.kstest(data, 'uniform')
print(stats.kstest(data, 'uniform'))

KstestResult(statistic=1.0, pvalue=0.0, statistic_location=1, statistic_sign=-1)


In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

class SparseTextVectorizer:

    def __init__(self, min_count=3, max_count=None,
                 sublinear_tf=False, use_idf=False,
                 lemmatizer=None, lowercase=False, remove_punctuation=False,
                 normalize_digits=False):
        self.min_count = min_count
        self.max_count = max_count
        self.sublinear_tf = sublinear_tf
        self.use_idf = use_idf
        self.lemmatizer = lemmatizer
        self.lowercase = lowercase
        self.remove_punctuation = remove_punctuation
        self.normalize_digits = normalize_digits

    def fit(self, texts):
        if self.lemmatizer is not None:
            texts = [[token.lemma_ for token in sent]
                     for sent in self.lemmatizer.pipe(texts)]
        self.word_indexer_ = TextCounter(self.min_count, self.max_count,
                                         lemmatizer=self.lemmatizer, lowercase=self.lowercase,
                                         remove_punctuation=self.remove_punctuation,
                                         normalize_digits=self.normalize_digits)
        self.word_indexer_.fit(texts)
        return self

    def transform(self, texts):
        # нужно преобразовать тексты в разреженную матрицу
        # A[i, j] = count <=> count(texts[j], self.words_[i]) = count > 0
        # X = np.zeros(shape=(len(texts), len(self.word_indexer_)), dtype=float)
        if self.lemmatizer is not None:
            texts = [[token.lemma_ for token in sent]
                     for sent in self.lemmatizer.pipe(texts)]
        rows, cols, values = [], [], []
        for i, text in enumerate(texts):
            word_counts = self.word_indexer_.transform(text)
            for index, count in word_counts.items():
                # X[i, index] = count
                rows.append(i)
                cols.append(index)
                if self.sublinear_tf:
                    count = np.log2(1.0+count)
                if self.use_idf:
                    count /= np.log2(1.0+self.word_indexer_.word_counts_.get(index, 1))
                values.append(count)
        return csr_matrix((values, (rows, cols)),
                          shape=(len(texts), len(self.word_indexer_)))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# lowercase+remove_punctuation 78.07
# lowercase+remove_punctuation+normalize_digits 73.01
# lowercase+remove_punctuation min_count=3 max_count=0.1 79.54
# lowercase+remove_punctuation min_count=3 max_count=0.1 sublinear_tf 80.63
# lowercase+remove_punctuation min_count=3 max_count=0.1 sublinear_tf,use_idf 82.06
# lemmatizer+lowercase+remove_punctuation min_count=3 max_count=0.1 sublinear_tf,use_idf 83.81
vectorizer = SparseTextVectorizer(
    lemmatizer=model, lowercase=True, remove_punctuation=True,
    normalize_digits=False, sublinear_tf=True, use_idf=True,
    min_count=3, max_count=0.1).fit(train_data)
X_train = vectorizer.transform(train_data)
X_valid = vectorizer.transform(valid_data)
#print(type(X_train), X_train.shape, X_test.shape)

cls = LogisticRegression()
cls.fit(X_train, train_labels)
pred_labels = cls.predict(X_valid)
print(" ".join(
    f"{100*x:.2f}" for x in f1_score(valid_labels, pred_labels, average=None)
))
print(f"{100*f1_score(valid_labels, pred_labels, average='macro'):.2f}")

In [ ]:
with open('test_lables.txt', 'w', encoding='utf-8') as f:
  for x in pred_labels:
    f.write(x+'\n')
files.download('test_lables.txt')